In [1]:
from sympy import *
import numpy as np
def func(fx):
    '''
    函数求导（1-2阶）
    '''
    diff_fx = diff(fx,x)
    diff2_fx = diff(fx,x,2)
    print 'fx:\t',fx
    print 'dfx:\t',diff_fx
    print 'd2fx:\t',diff2_fx
    return fx,diff_fx,diff2_fx

def fval(fx,VarsName,VarsValue):
    '''函数赋值运算
    fx -- symbol function
    VarsName -- list,eg.['x','y']
    VarsValue -- list,eg.np.array([1,2])
    '''
    try:
        Vars = zip(VarsName,VarsValue)
    #     print 'fx',fx,'VarsN:',VarsName,'VarsV:',VarsValue
        fxValue = fx
        for VarName,VarValue in Vars:
            fxValue = fxValue.subs(VarName,VarValue)
        return fxValue
    except:
        return fx.subs(VarsName,VarsValue)

def matval(mat,VarsName,VarsValue):
    '''矩阵符号赋值运算程序
        mat -- list, eg.[[x+y],[x-y]]
        VarsNmae -- list,eg.['x','y']
        VarsValue -- np.array.eg.np.array([1,2])
    '''
    res = []
    for row in range(len(mat)):
        temp = []
        if np.size(mat[row]) > 1:
            for col in range(len(mat[row])):
                temp.append(float(fval(mat[row][col],VarsName,VarsValue)))
            res.append(temp)
        else:
            res.append(float(fval(mat[row],VarsName,VarsValue)))
    return res
    
def singlejac(f,vars):
    '''雅克比矩阵'''
    jac = []
    try:
        for var in vars:
            jac.append(diff(f,var))
        return jac
    except:
        return diff(f,vars)
        
    
def jacobian(f,vars):
    if type(f) == list:
        listJac = []
        for func in f:
            listJac.append(singlejac(func,vars))
        return listJac
    else:
        return singlejac(f,vars)

def hess(f,vars):
    '''海森矩阵'''
    hess = []
    for i in range(len(f)):
        hess.append(jacobian(f[i],vars))
    return hess

def find(x,cond,num,start='left'):
    '''查找符合条件的数值坐标和位置
    x -- np.array, extracted array
    cond -- str, extracted condition
    num -- number of data
    start -- beginning
    '''
    nWhere = []
    nExtr = []
    y = np.where(eval('x'+cond))
    if start == 'left':
        for row in range(len(y)):
            nWhere.append(y[row][:num])
    
    elif start == 'right':
        for row in range(len(y)):
            nWhere.append(y[row][-num:])
    if len(y) == 1:
        nExtr = [x[ nWhere[i]] for i in range(np.size(nWhere,0))]
    elif len(y) == 2:
        nExtr = [x[nWhere[0][i],nWhere[1][i]] for i in range(np.size(nWhere,1))]
    return nWhere,nExtr

def ArrSort(x):
    sortx = np.sort(x)
    sortxID = [int(np.argwhere(x==i)[0,0]) for i in sortx]
    return sortx,sortxID
if __name__ == '__main__':
    x = var('x')
    fx = x*x + 2*x + 1
    print 'DERIVATIVA CHECK(func):'
    fx,df,d2f = func(fx)
    x = var('t s')
    fx = (t-4)*(t-4)*(t-4) + (s+2)*(s+2) + 1
    print '(SUBS CHECK(fval)):'
    print fval(fx,x,[4,-2])
    
    x = var('t s e')
    fx = [(t-3)*(t-3)*(t-3) + (s-2)*s +e*(e-1),t+s+e]
    print '(Jac CHECK(Jacobian)):'
    jac = np.array(jacobian(fx,x))
    print jac,type(jac)
    print '(JAC VALUE CHECK(matval)):'
    JacValue  = matval(jac,x,[1,2,3])
    print JacValue,type(JacValue)
    
    x = var('t s e')
    fx = (t-3)*(t-3)*(t-3) + (s-2)*s +e*(e-1)
    jac = np.array(jacobian(fx,x))
    print '(Hess CHECK(hess)):'
    hessian = hess(jac,x)
    print hessian,type(hessian)
    print '(HESS VALUE CHECK(matval)):'
    HessValue  = matval(jac,x,[1,2,3])
    print HessValue,type(HessValue)
    print '(find check(find)):'
    x1 = np.array([1,2,3,4,5,6,7,8,9])
    y1,y1_value = find(x1,'!=3',10,'right')
    print 'y1:',y1
    print 'y1_value:',y1_value
    x2 = np.arange(9).reshape(3,3)
    y2,y2_value = find(x2,'>3',10,'right')
    print 'y2:',y2
    print 'y2_value:',y2_value
    print '(Array sort(ArrSort):'
    a = [2,3,4,1,7,6]
    asort,ID = ArrSort(a)
    print 'a:',a
    print 'asort:',asort
    print 'ID:',ID

DERIVATIVA CHECK(func):
fx:	x**2 + 2*x + 1
dfx:	2*x + 2
d2fx:	2
(SUBS CHECK(fval)):
1
(Jac CHECK(Jacobian)):
[[3*(t - 3)**2 2*s - 2 2*e - 1]
 [1 1 1]] <type 'numpy.ndarray'>
(JAC VALUE CHECK(matval)):
[[12.0, 2.0, 5.0], [1.0, 1.0, 1.0]] <type 'list'>
(Hess CHECK(hess)):
[[6*t - 18, 0, 0], [0, 2, 0], [0, 0, 2]] <type 'list'>
(HESS VALUE CHECK(matval)):
[12.0, 2.0, 5.0] <type 'list'>
(find check(find)):
y1: [array([0, 1, 3, 4, 5, 6, 7, 8], dtype=int64)]
y1_value: [array([1, 2, 4, 5, 6, 7, 8, 9])]
y2: [array([1, 1, 2, 2, 2], dtype=int64), array([1, 2, 0, 1, 2], dtype=int64)]
y2_value: [4, 5, 6, 7, 8]
(Array sort(ArrSort):
a: [2, 3, 4, 1, 7, 6]
asort: [1 2 3 4 6 7]
ID: [3, 0, 1, 2, 5, 4]


# 黄金分割

In [7]:
def minHJ(f,a,b,vars,eps=1e-6):
    l = a + 0.382*(b-a)
    u = b + 0.618*(b-a)
    k = l
    tol = b - a
    while tol > eps and k <100000:
        fl = fval(f,vars,[a])
        fu = fval(f,vars,[b])
        if fl > fu:
            a = l
            l = u
            u = a + 0.618*(b-a)
        else:
            b = u
            u = l
            l = a + 0.618*(b-a)
        k += 1
        tol = abs(b-a)
        if k>=100000:
            break
    minf = fval(f,vars,[(a+b)/2.])
    return (a+b)/2.,minf
if __name__ == '__main__':
    t = var('t')
    ft = (t*t-1)*(t*t-1) + (t-1)*(t-1) + 3
    x,minf = minHJ(ft,-10.,10.,['t'])
    print 'x:',x
    print 'minf:',minf

x: 1.00004822329
minf: 3.00000001162788


# 进退法

In [8]:
def minJT(f,x0,h0,vars,eps=1e-6):
    x1 = x0
    k = 0
    h = h0
    while 1:
        x4 = x1 + h
        k += 1
        f4 = fval(f,vars,[x4])
        f1 = fval(f,vars,[x1])
        if f4 < f1:
            x2 = x1
            x1 = x4
            f2 = f1
            f1 = f4
            h = 2*h
        else:
            if k == 1:
                h = -h
                x2 = x4
                f2 = f4
            else:
                x3 = x2
                x2 = x1
                x1 = x4
                break
    minx = min(x1,x3)
    maxx = x1 + x3 - minx
    return minx,maxx
if __name__ == "__main__":
    t = var('t')
    ft = (t*t-1)*(t*t-1) + (t-1)*(t-1) + 3
    x1,x2 = minJT(ft,0.,0.1,['t'])
    print x1,x2

0.3 1.5


# 全局牛顿法(程序有BUG,在极值点)

In [9]:
import numpy as np
from sympy import *

def f():
    t = var('t')
    ft = pow(t,2) - log(t) - 5
    diff_ft = diff(ft,t)
    diff2_ft = diff(ft,t,2)
    print ft
    print diff_ft
    print diff2_ft
    return ft,diff_ft,diff2_ft

def minGLN(x0,eps):
    bConti = 1
    fx,df,d2f = f()
    k = 1
    while bConti:
        fx0 = fx.subs(t,x0)
        dfx = df.subs(t,x0)
        d2fx = d2f.subs(t,x0)
        if dfx == 0:
            if d2fx >= 0:
                x = x0
                bConti = 0
            delta = eps
            while 1:
                fd = fx.subs(t,x0+delta)
                if fd >= fx0:
                    delta = delta * 2
                    continue
                else:
                    x0 = x0 + delta
                    break
        else:
            beta = d2fx
            if beta <= 0:
                beta = 1
            alpha = 1
            while 1:
                
                x1 = x0 -alpha * dfx/beta
                fx1 = fx.subs(t,x1)
                tol = fx1 - fx0 + dfx*dfx*alpha/4./beta
                print '第 %d 次循环 x值： %f' %(k,x1)
                k += 1
                if tol <= 0:
                    if abs(x1 - x0) <= eps:
                        bConti = 0
                        x = x1
                        break
                    else:
                        x0 = x1
                        break
                else:
                    alpha = alpha/2
                    continue
    minf = fx.subs(t,x)
    print 'x值：%f  最小值： %f' %(x,minf)

if __name__ == "__main__":
    minGLN(1.1,1e-6)

t**2 - log(t) - 5
2*t - 1/t
2 + t**(-2)
第 1 次循环 x值： 0.643275
第 2 次循环 x值： 0.703954
第 3 次循环 x值： 0.707100
第 4 次循环 x值： 0.707107
第 5 次循环 x值： 0.707107
第 6 次循环 x值： 0.707107
x值：0.707107  最小值： -4.153426


# 割线法

In [10]:
import numpy as np
from sympy import *
def f():
    t = var('t')
    ft = pow(t,2) - log(t) - 5
    diff_ft = diff(ft,t)
    diff2_ft = diff(ft,t,2)
    print ft
    print diff_ft
    print diff2_ft
    return ft,diff_ft,diff2_ft

def minGX(x0,x1,eps):
    fx,df,d2f = f()
    k = 0
    tol = 1
    while tol > eps:
        print '第 %d 次循环 x0: %f x1: %f' %(k+1,x0,x1)
        dfx0 = df.subs(t,x0)
        dfx1 = df.subs(t,x1)
        x2 = x1 -((x1 - x0)*dfx1)/(dfx1-dfx0)
        tol = abs(dfx1)
        k += 1
        x0 = x1
        x1 = x2
    x = x2
    minf = fx.subs(t,x)
    print 'x值: %f 最小值: %f' %(x,minf)
if __name__ == "__main__":
    minGX(5,8,1e-6)        

t**2 - log(t) - 5
2*t - 1/t
2 + t**(-2)
第 1 次循环 x0: 5.000000 x1: 8.000000
第 2 次循环 x0: 8.000000 x1: 0.160494
第 3 次循环 x0: 0.160494 x1: 2.287197
第 4 次循环 x0: 2.287197 x1: 1.411455
第 5 次循环 x0: 1.411455 x1: 0.496027
第 6 次循环 x0: 0.496027 x1: 0.794705
第 7 次循环 x0: 0.794705 x1: 0.721728
第 8 次循环 x0: 0.721728 x1: 0.706263
第 9 次循环 x0: 0.706263 x1: 0.707115
第 10 次循环 x0: 0.707115 x1: 0.707107
x值: 0.707107 最小值: -4.153426


# Hook-Jeeves

In [11]:
def minPS(f,x0,delta,gamma,sita,var,eps):
    k = 0
    n = len(var)
    tol =0
    print ('k\tyf\ty[0]\ty[1]\tdelta[0]\tdelta[1]\ttol')
    while 1:
        k += 1
        y = x0
        yf = f.subs(t,y[0]).subs(s,y[1])
        #探测移动
        for i in range(n):
            tempy = [0,0]
            tempy[i] = delta[i]
            tempf = f.subs(t,y[0]+tempy[0]).subs(s,y[1]+tempy[1])
            if tempf < yf:
                y = y + tempy
            else:
                tempf = f.subs(t,y[0]-tempy[0]).subs(s,y[1]-tempy[1])
                if tempf < yf:
                    y = y - tempy
        x1 = y
        fx1 = f.subs(t,x1[0]).subs(s,x1[1])
        tol = np.linalg.norm(delta)
        #模式移动
        if fx1 < yf:
            y = x1 + gamma*(x1-x0)
        else:

            if tol < eps:
                x = x0
                break
            else:
                if (x1 != x0).all():
                    y = x1
                else:
                    y = x1
                    delta = sita * delta
        x0 = x1

        print ('%d\t%5.3f\t%5.3f\t%5.3f\t%10.7f\t%10.7f\t%10.7f' %(k,yf,y[0],y[1],delta[0],delta[1],tol))
        if k == 100:
            break
    minf = f.subs(t,y[0]).subs(s,y[1])
    print ('x:%f   %f\tminf:%f' %(y[0],y[1],minf))
if __name__ == '__main__':
    x = var('t s')
    fx = 3*t*t + s*s -2*t*s + 4*t + 3*s
    minPS(fx,np.array([-2,6]),np.array([0.2,0.2]),1.4,0.2,[t,s],1e-6)

k	yf	y[0]	y[1]	delta[0]	delta[1]	tol
1	82.000	-1.520	6.480	 0.2000000	 0.2000000	 0.2828427
2	81.880	-1.320	5.720	 0.2000000	 0.2000000	 0.2828427
3	74.480	-1.120	5.520	 0.2000000	 0.2000000	 0.2828427
4	67.560	-0.920	5.320	 0.2000000	 0.2000000	 0.2828427
5	61.120	-0.720	5.120	 0.2000000	 0.2000000	 0.2828427
6	55.160	-0.520	4.920	 0.2000000	 0.2000000	 0.2828427
7	49.680	-0.320	4.720	 0.2000000	 0.2000000	 0.2828427
8	44.680	-0.120	4.520	 0.2000000	 0.2000000	 0.2828427
9	40.160	0.080	4.320	 0.2000000	 0.2000000	 0.2828427
10	36.120	0.280	4.120	 0.2000000	 0.2000000	 0.2828427
11	32.560	0.480	3.920	 0.2000000	 0.2000000	 0.2828427
12	29.480	0.680	3.720	 0.2000000	 0.2000000	 0.2828427
13	26.880	0.880	3.520	 0.2000000	 0.2000000	 0.2828427
14	24.760	0.600	3.320	 0.2000000	 0.2000000	 0.2828427
15	22.920	0.600	3.120	 0.2000000	 0.2000000	 0.2828427
16	21.160	0.120	2.920	 0.2000000	 0.2000000	 0.2828427
17	19.360	0.400	2.720	 0.2000000	 0.2000000	 0.2828427
18	17.680	0.400	2.520	 0.2000

# Rosenbrock

In [12]:
def minRB(f,x0,D,delta,alpha,beta,var,eps):
    k = 0
    x0 = x0.T
    y0 =fval(fx,var,x0)
    print ('y0:%f' %y0)
    delta0 = delta
    n = len(var)
    Q,R = np.linalg.qr(D)
    y = x0
    print ('k\ty[0]\ty[t]\ttol')
    tol = 1
    while 1:
        yf = fval(fx,var,y)
        bconti = 1
        while bconti:
            for i in range(n):
                tmpy = delta[i]*Q[:,i]
                tmpf = fval(f,var,y+tmpy)
                bconti = 0
                if tmpf <= yf:
                    y = y + tmpy
                    delta[i] = alpha + delta[i]
                    bconti = 1
                else:
                    delta[i] = -beta * delta[i]
        yfn = fval(f,var,y)
        if yfn < yf:
            continue
        else:
            if yfn == yf:
                if yfn < fval(f,var,x0):
                    x1 = y
                    tol = np.linalg.norm(x1-x0)
                    if tol < eps:
                        x = x1
                        break
                    else:
                        D = Q
                        D[:,1] = x1 - x0
                        Q,R = np.linalg.qr(D)
                    delta = delta0
                    x0 = x1
                    y = x0
                else:
                    if max([abs(i) for i in delta]) < eps:
                        x = x0
                        break
                    else:
                        continue
        k += 1
        print ('%d\t%f\t%f\t%f' %(k,y[0],y[1],tol))
        if k == 1000:
            break
    minf = fval(f,var,x)
    print 'x:',x
    print ('minf:%f' %minf)

if __name__ == '__main__':
    x = var('t s')
    fx = 3*t*t + s*s - 2*t*s + 4*t + 3*s
    minRB(fx,np.array([0,0]),np.array([[1,2],[0,1]]),[1,1],3,0.5,['t','s'],1e-6)

y0:0.000000
k	y[0]	y[t]	tol
1	-2.156250	-3.687500	4.271659
2	-1.742188	-3.203125	0.637234
3	-1.728851	-3.230347	0.030313
4	-1.752393	-3.236153	0.024247
5	-1.740766	-3.236153	0.011627
6	-1.740766	-3.242001	0.005848
7	-1.746648	-3.242001	0.005882
8	-1.746648	-3.247849	0.005848
9	-1.748110	-3.247849	0.001462
10	-1.748110	-3.248214	0.000365
11	-1.749574	-3.248214	0.001464
12	-1.749574	-3.249679	0.001465
13	-1.749940	-3.249679	0.000366
14	-1.749940	-3.250045	0.000366
15	-1.750032	-3.250045	0.000092
16	-1.750032	-3.250034	0.000011
17	-1.750021	-3.250034	0.000011
18	-1.750021	-3.250022	0.000011
19	-1.750009	-3.250022	0.000011
20	-1.750009	-3.250011	0.000011
21	-1.750006	-3.250011	0.000003
22	-1.750006	-3.250008	0.000003
23	-1.750003	-3.250008	0.000003
24	-1.750003	-3.250005	0.000003
25	-1.750001	-3.250005	0.000003
26	-1.750001	-3.250002	0.000003
x: [-1.75000086 -3.25000216]
minf:-8.375000


# SimpleSearch

In [13]:
def minSimpSearch(f,X,alpha,sita,gamma,beta,var,eps):
    '''X初始值会影响优化结果
    节点围成的形状，涵盖极值'''
    n = np.size(X,1)
    FX = np.zeros(n)
    k = 0
    while 1:
        k += 1
        print '\n',k
        for i in range(n):
            FX[i] = fval(f,var,X[:,i])                  #计算各节点对应的数值
        for i in FX:                                   #避免出现两个节点计算数值一致
            if list(FX).count(i) > 1:
                FX[list[FX].index(i)] += i/10000000.
            else:
                break
        XS = np.sort(FX)                               #对计算结果进行排序
        IX = [int(np.argwhere(FX==i)) for i in XS]    #获取序号
        Xsorted = []                                   #形成排序后的X
        for i in IX:
            Xsorted.append(list(X[:,i]))
        Xsorted = np.array(Xsorted).T
        print 'X      :\n',X
        print 'Xsorted:\n',Xsorted
        px = [sum(Xsorted[i,:])/n for i in range(n-1)]#计算形心的位置
        Fpx = fval(f,var,px)
        print 'px:',px
        SumF = 0
        for i in range(n):
            SumF =SumF + pow((FX[IX[i]]-Fpx),2)
        SumF = sqrt(SumF/n)                            #计算各节点距离形心的数值差
  
        if SumF <= eps:
            x = Xsorted[:,0]
            break
        else:
            x2 = px + alpha*(px - Xsorted[:,-1])
            fx2 = fval(f,var,x2)
            if fx2 < XS[0]:                          #小于第一个点，则在中点和数值最大点之间选点
                x3 = px + gamma *(x2 - px)
                fx3 = fval(f,var,x3)
                if fx3 < XS[0]:
                    Xsorted[:,-1] = x3
                    X = Xsorted
                    continue
                else:
                    Xsorted[:,-1] = x2
                    X = Xsorted
                    continue
            else:                                    #小于第一个点，则在中点和数值最大点之间选点
                if fx2 < XS[n-2]:
                    Xsorted[:,-1] = x2
                    X = Xsorted
                    continue
                else:
                    if fx2 < XS[-1]:
                        Xsorted[:,-1] = x2
                    x4 = px + beta*(Xsorted[:,-1] - px)
                    fx4 = fval(f,var,x4)
                    FNnew = fval(f,var,Xsorted[:,n-2])
                    
                    if fx4 < FNnew:
                        Xsorted[:,-1] = x4
                        X = Xsorted
                        continue
                    else:
                        x0 = Xsorted[:,0]
                        for j in range(n):
                            Xsorted[:,j] = x0 + sita*(Xsorted[:,j]-x0)
        if k == 100:
            break
            
        X = Xsorted
    minf = fval(f,var,x)
    print 'X_final:',x
    print 'minf:',minf
if __name__ == '__main__':

    x = var('t s')
    fx = 3*t*t + s*s - t*s + 3*s -5
    X = np.array([[-1.,1,8],[-1,4,8]])
    minSimpSearch(fx,X,1.2,0.5,2,0.3,['t','s'],1e-6)


1
X      :
[[-1.  1.  8.]
 [-1.  4.  8.]]
Xsorted:
[[-1.  1.  8.]
 [-1.  4.  8.]]
px: [2.6666666666666665, 3.6666666666666665]

2
X      :
[[-1.          1.          0.74666667]
 [-1.          4.          2.10666667]]
Xsorted:
[[-1.          0.74666667  1.        ]
 [-1.          2.10666667  4.        ]]
px: [0.24888888888888885, 1.7022222222222221]

3
X      :
[[-1.          0.74666667 -0.65244444]
 [-1.          2.10666667 -1.05511111]]
Xsorted:
[[-0.65244444 -1.          0.74666667]
 [-1.05511111 -1.          2.10666667]]
px: [-0.30192592592592593, 0.01718518518518497]

4
X      :
[[-0.65244444 -1.         -0.67941926]
 [-1.05511111 -1.         -0.73502815]]
Xsorted:
[[-0.65244444 -0.67941926 -1.        ]
 [-1.05511111 -0.73502815 -1.        ]]
px: [-0.77728790123456781, -0.93004641975308677]

5
X      :
[[-0.65244444 -0.67941926 -0.24277886]
 [-1.05511111 -0.73502815 -0.76215783]]
Xsorted:
[[-0.24277886 -0.65244444 -0.67941926]
 [-0.76215783 -1.05511111 -0.73502815]]
px: [-0.52488

# POWELL

In [14]:
def minPowell(f,x0,P,vars,eps=1e-6):
    '''
    P-方向矢量
    '''
    n = len(vars) + 1
    z = var('z')
    FY = []
    k = 0
    for j in range(n):
        FY.append([])
    while 1:
        k += 1
        y = np.zeros(P.shape)
        y[:,0] = x0
        for i in range(n-1):
            yv = y[:,i] + z*P[:,i]
            fy = fval(f,vars,yv)
            a,b = minJT(fy,0,0.1,['z'])
            tz,HJminf = minHJ(fy,a,b,['z'])
            y[:,i+1] = y[:,i] + tz*P[:,i]
        P[:,-1] = y[:,-1] - y[:,0]
        NormP = np.linalg.norm(P[:,-1])
        if NormP <= eps:
            x = y[:,-1]
            break
        else:
            for j in range(n):
                FY[j] = fval(f,vars,y[:,j])
            maxDF = -1E32
            m = 0
            for j in range(n-1):
                df = FY[j] - FY[j+1]
                if df > maxDF:
                    maxDF = df
                    m = j + 1
            tmpF = fval(f,vars,2*y[:,-1]-y[:,0])
            fl = FY[0] - 2*FY[-1] + tmpF
            if fl < 2*maxDF:
                yv = y[:,-1] + z*P[:,-1]
                fy = fval(f,vars,yv)
                a,b = minJT(fy,0,0.1,['z'])
                tz,HJminf = minHJ(fy,a,b,['z'])
                x0 = y[:,-1] + tz*P[:,-1]
            else:
                x0 = y[:,-1]
        if k == 10:
            x = y[:,-1]
            break
    minf = fval(f,vars,x)
    return y[:,-1],minf
if __name__ == '__main__':
    x = var('t s')
    fx = 3*t*t + s*s - t*s + 3*s - 5
    P = np.array([[-1.,-1,1],[-2,3,-3]])
    x,minf = minPowell(fx,[0.,0],P,['t','s'])
    print 'x:',x
    print 'minf:',minf

x: [-0.27280071 -1.63629643]
minf: -7.45454542891543


# 快速下降

In [15]:
def minFD(f,x0,vars,eps=1e-6):
    z = var('z')
    tol = 1
    k = 0
    print ('k\ttol\tx')
    while tol > eps:
        k += 1
        v = []
        gradf = -np.array(jacobian(f,vars))
        for i in range(len(gradf)):
            v.append(fval(gradf[i],vars,x0))
        v = np.array([float(i) for i in v])
        tol = np.linalg.norm(v)
        y = x0 + z*v
        yf = fval(f,vars,y)
        a,b = minJT(yf,0,0.1,['z'])
        xm,HJminf = minHJ(yf,a,b,['z'])
        x1 = x0 + xm*v
        x0 = x1
        print ('%d\t%f' %(k,tol)),
        print x1
    x = x1
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    fx = 3*t*t + s*s - t*s + 3*s -5
    x,minf = minFD(fx,[1.,-3.],['t','s'])
    print 'x:',x
    print 'minf:',minf  

k	tol	x
1	9.848858 [-0.4827001  -2.34102218]
2	1.321609 [-0.19600316 -1.72167476]
3	0.599100 [-0.28589693 -1.68092582]
4	0.083405 [-0.26804632 -1.64157571]
5	0.036564 [-0.2735322  -1.63908711]
6	0.005097 [-0.27244114 -1.63668229]
7	0.002235 [-0.27277648 -1.63653015]
8	0.000312 [-0.27270978 -1.63638312]
9	0.000137 [-0.27273029 -1.63637382]
10	0.000019 [-0.2727262  -1.63636485]
11	0.000008 [-0.27272746 -1.63636427]
12	0.000001 [-0.2727272  -1.63636373]
13	0.000001 [-0.27272729 -1.63636368]
x: [-0.27272729 -1.63636368]
minf: -7.45454545454545


# 共轭梯度

In [16]:
def minGETD(f,x0,vars,eps=1e-6):
    x0 = x0.T
    n = len(vars)
    z = var('z')
    gradf = np.array(jacobian(f,vars))
    v0 = []
    for i in range(len(gradf)):
        v0.append(fval(gradf[i],vars,x0))
    v0 = np.array([float(i) for i in v0])
    p = -v0.T
    k = 0

    while 1:
        v = []
        for i in range(len(gradf)):
            v.append(fval(gradf[i],vars,x0))
        v = np.array([float(i) for i in v])
        tol = np.linalg.norm(v)
        if tol <= eps:
            x = x0
            break
        y = x0 + z*p #确定合适的步长
        yf = fval(f,vars,y)
        a,b = minJT(yf,0,0.1,['z'])
        xm,HJminf = minHJ(yf,a,b,['z'])
        x1 = x0 + xm*p
        vk = []
        for i in range(len(gradf)):
            vk.append(fval(gradf[i],vars,x1))
        vk = np.array([float(i) for i in vk])
        tol = np.linalg.norm(vk)
        if tol <= eps:
            x = x1
            break
        if k+1 == n:
            x0 = x1
            continue
        else:
            lamda = np.dot(vk,vk)/np.dot(v,v) #共轭方向
            print 'lamda:',lamda
            p = vk.T + lamda*p
            k = k+1
            x0 = x1
        print 'k:',k
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s e')
    fx = (t-3)*(t-3) + (s-2)*s +e*(e-1)
    x,minf = minGETD(fx,np.array([-100.,6.,7.]),['t','s','e'])
    print 'x:',x
    print 'minf:',minf    
            

lamda: 7.22607010046e-05
k: 1
lamda: 2.65528991923e-11
k: 2
x: [ 3.00000001  1.          0.5       ]
minf: -1.25000000000000


# 牛顿法

In [17]:
def minNT(f,x0,vars,eps=1e-6):
    tol = 1
    x0 = x0.T
    k = 0
    gradf = np.array(jacobian(f,vars))
    jacf = np.array(hess(gradf,vars))
    while tol > eps:
        k += 1

        v = matval(gradf,vars,x0)
        tol = np.linalg.norm(v)
        v = np.array(v)
        pv = matval(jacf,vars,x0)
        pv = np.matrix(pv)
        p = np.dot(-pv.I,v.T)
        x1 = np.array(x0 + p)
        x0 = x1[0]
    x = x1[0]
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    fx = s**2 + t**2 + 8.0/(s + t - 1.0) + 8.0/(-s + 2*t - 2)
    x,minf = minNT(fx,np.array([-3.,-1.]),['t','s'])
    print 'x:',x
    print 'minf:',minf        

x: [ 2.44794397  0.44698515]
minf: 13.6807883141640


# 修改牛顿法

In [18]:
def minMNT(f,x0,vars,eps=1e-6):
    tol = 1
    x0 = x0.T
    z = var('z')
    k = 0 
#     print 'k\ttol\tx'
    while tol > eps:
        k += 1
        gradf = jacobian(f,vars)
        jacf = hess(gradf,vars)
        v = matval(gradf,vars,x0)
        tol = np.linalg.norm(v)
        pv = matval(jacf,vars,x0)
        pv = np.matrix(pv)
        v = np.array(v)
        p = np.dot(-pv.I,v.T)
        y = x0 + z*p
        y = np.array(y)
        yf = fval(f,vars,y[0])
        a,b = minJT(yf,0,0.1,['z'])
        xm,HJminf = minHJ(yf,a,b,['z'])
        x1 = np.array(x0 + xm*p)
        x0 = x1[0]
#         print '%d\t%f' %(k,tol),
#         print x1[0]
    x = x1[0]
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s l')
    fx = t*t - t*s + s*s + 2 +(l-1)*(l-1)
    x,minf = minMNT(fx,np.array([1.,3,2]),x)
    print 'x:',x
    print 'minf:',minf

x: [  5.20908512e-09   1.56272554e-08   1.00000001e+00]
minf: 2.00000000000000


# 拟牛顿法（DFP）

In [19]:
def minDFP(f,x0,vars,eps=1e-6):
    x0 = x0.T
    n = len(vars)
    z = var('z')
    H = np.eye(n,n)
#     print 'H:',H
    gradf = jacobian(f,vars)
#     print 'gradf:',gradf
    v0 = np.array(matval(gradf,vars,x0))
#     print 'v0',v0,type(v0)
    p = np.dot(-H,v0.T)
#     print 'p:',p
    k = 0
    while 1:
        v = matval(gradf,vars,x0)
        tol = np.linalg.norm(v)
        print '%d\t%f\t%f' %(k,tol,fval(f,vars,x0)),
        print x0
        if tol <= eps:
            x = x0
            break
#         print 'x0:',x0
#         print 'z:',z
#         print 'p:',p
        y = x0 + z*p
#         print 'y:',np.size(y)
        yf = fval(f,vars,y)
        a,b = minJT(yf,0,0.1,['z'])
        xm,HJminf = minHJ(yf,a,b,['z'])
#         print 'xm:',xm
        x1 = x0 + xm*p
#         print 'x1:',x1
        vk = np.array(matval(gradf,vars,x1))
#         print 'vk',vk
        tol = np.linalg.norm(vk)
#         print 'tol:',tol

        if tol <= eps:
            x = x1
            break
        if k+1 == n:
            x0 = x1
            k += 1
            continue
        else:
            dx = x1 - x0;dxT = np.transpose([dx]) 
            dx = np.transpose(dxT) 
            
#             print 'dx:\n',dx
#             print 'dxT:\n',dxT
            dgf = vk - v
            dgf = np.transpose([dgf])
            dgfT = np.transpose(dgf)
#             dgfT = np.transpose(dgf)
#             print 'dgf:\n',dgf
#             print 'dgfT:\n',dgfT
            mdx = np.dot(dx , dxT)
            mdgf = np.dot(dgf , dgfT)
#             print 'mdx:\n',mdx
#             print 'mdgf:\n',mdgf
            fz = np.dot(np.dot(np.dot(H,dgf),dgfT),H)         
#             print 'fz:\n',fz
            
#             print '!!!!!!!!\n',mdx,np.matrix(np.dot(dxT,dgfT)),mdx/np.matrix(np.dot(dxT,dgfT))
            H = H + mdx/np.matrix(np.dot(dxT,dgfT)) - fz/np.dot(dgfT,np.matrix(np.dot(H,dgf))) #这里与原文不一致
#             print 'H:',H
            p = np.dot(-H,vk.T)
            p = np.array(p.tolist()[0])
            k += 1
            x0 = x1
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    fx = t*t - t*s + s*s + 2
    x,minf = minDFP(fx,np.array([-1000,-1000]),x)
    print 'x:',x
    print 'minf:',minf

0	1414.213562	1000002.000000 [-1000 -1000]
1	58.881009	1735.486584 [ 41.63516043  41.63516043]
2	0.000686	2.000000 [-0.00048527 -0.00048527]
3	1.310330	2.858482 [ 0.92654317  0.92654317]
4	0.000015	2.000000 [ -1.07990273e-05  -1.07990273e-05]
x: [ -9.32660097e-10  -9.32660097e-10]
minf: 2.00000000000000


# Rosen梯度投影法

In [20]:
def minRosen(f,A,b,x0,vars,eps=1e-6):
    z = var('z')
    x0 = np.transpose([x0])
    n = len(vars)
    m = np.size(A,0)
    gf = jacobian(f,vars)
    bConti = 1
    kk = 0
    tol = 1
    print 'k\ttol\tx'
    while bConti:
        kk += 1
        print '%d\t%f\t%f' %(kk,tol,fval(f,vars,x0)),
        print x0.T
        k = 0
        s = 0
        A1 = A
        A2 = A
        b1 = b
        b2 = b
        for i in range(m):
            dfun = A[i,:] * x0 - b[i]
            if abs(dfun) < 1e-8:
                A1[k,:] = A[i,:]
                b1[k,0] = b[i]
                k += 1
            else:
                A2[s,:] = A[i,:]
                b2[s,0] = b[i]
                s += 1
        if k > 0:
            A1 = A1[:k,:]
            b1 = b1[:k,:]
        if s > 0:
            A2 = A2[:s,:]
            b2 = A2[:s,:]
        while 1:
            P = np.eye(n,n)
            if k > 0:
                tM = A1.T
                P = P- tM*(A1*tM).I*A1             #投影矩阵
            gv = np.matrix(matval(gf,vars,x0)).T
            d = -P * gv
            if d.all() ==0:
                if k == 0 :
                    x = x0
                    bConti = 0
                    break
                else:
                    w = (A1*tM).I*A1*gv
                    if w>= 0:
                        x = x0
                        bConti = 0
                        break
                    else:
                        u,index = min(w)
                        sA1 = np.size(A1)
                        if sA1[0] == 1:
                            k = 0
                        else:
                            k = sA1[1]
                            A1 = np.array([A1[:index-1,:],A1[index+1:sA2[1],:]]).T
            else:
                break
        yl = x0 + z*d
        yl_list = yl.T
        yl_list = yl_list.tolist()
#         print 'yl_list[0]:',yl_list[0]
        tmpf = fval(f,vars,yl_list[0])
        bb = b2 - A2*x0
        dd = A2*d
        if dd.all() >= 0:
#             print 'tmpf:',tmpf
            tmpI,lm = minJT(tmpf,0,0.1,['z'])
        else:
            lm = 1e32
            for i in range(len(dd)):
                if dd[i] < 0:
                    if bb[i]/dd[i] < lm:
                        lm = bb[i]/dd[i]
        xm,minf = minHJ(tmpf,0,lm,['z'],1e-14)
        tol = np.linalg.norm(xm*d)
        if tol < eps:
            x = x0
            break
        x0 = x0 + xm*d
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    fx = 2*t*t - 2*t*s + s*s + 3*t - 8*s + 2
    x,minf = minRosen(fx,np.matrix([[-1.,1],[-3,-5]]),np.array([[-4],[-8]]),np.array([0,0]),x)
    print 'x:',x
    print 'minf:',minf
        

k	tol	x
1	1.000000	2.000000 [[0 0]]
2	2.398842	-8.248077 [[-0.84228955  2.24610547]]
3	2.246171	-13.759514 [[ 1.26083754  3.03486315]]
4	1.278503	-16.871309 [[ 0.74268906  4.2036626 ]]
5	1.216902	-18.490232 [[ 1.85553135  4.69603647]]
6	0.665389	-19.334429 [[ 1.58542474  5.30413604]]
7	0.633851	-19.773645 [[ 2.16470362  5.561436  ]]
8	0.346187	-20.002165 [[ 2.02416829  5.87781477]]
9	0.329781	-20.121058 [[ 2.32555366  6.01168952]]
10	0.180112	-20.182915 [[ 2.25243707  6.17629315]]
11	0.171577	-20.215097 [[ 2.40924005  6.24594467]]
12	0.093708	-20.231841 [[ 2.37119939  6.33158369]]
13	0.089267	-20.240552 [[ 2.45277996  6.36782157]]
14	0.048754	-20.245085 [[ 2.43298837  6.41237734]]
15	0.046443	-20.247443 [[ 2.47543263  6.43123098]]
16	0.025365	-20.248669 [[ 2.46513558  6.4544122 ]]
17	0.024163	-20.249308 [[ 2.48721823  6.46422124]]
18	0.013197	-20.249640 [[ 2.48186093  6.47628185]]
19	0.012572	-20.249813 [[ 2.49334998  6.48138522]]
20	0.006866	-20.249903 [[ 2.49056272  6.48766003]]
21	0

# 外罚函数

In [21]:
def minPF(f,x0,A,b,cl,p,vars,eps=1e-6):
    k = 0
    FE = 0
    print 'k\ttol\tx'
    for i in range(len(b)):
        FE = FE + pow(vars*A[0,:].T - b[i],2)
    print 'FE:',FE
    x1 = x0.T
    x2 = 1e32
    tol = 1
    while 1:
        k += 1

        M = cl*p
        FF = M*FE
        Sumf = f + FF
        Sumf = Sumf.tolist()
        Sumf_func = Sumf[0][0]
#         print 'Sumf',Sumf[0][0]
        x2,minf = minNT(Sumf_func,x1.T,vars)
        tol = np.linalg.norm(x2-x1)
        print '%d\t%f\t%f' %(k,tol,fval(f,vars,x1)),
        print x1.T
#         print 'x2',x2
#         print 'x1',x1
#         print 'tol',tol
        if tol <= eps:
            x = x2  
            break
        else:
            cl = M
            x1 = x2
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = 0.5*t*t + s*s/4.
    A = np.matrix([1.,1]);b = np.array([1])
    cl = 0.05; p = 2.5
    x,minf = minPF(f,np.array([0.,0]),A,b,cl,p,x)
    print 'x:',x
    print 'minf:',minf

k	tol	x
FE: [[(1.0*s + 1.0*t - 1)**2]]
1	0.319438	0.000000 [ 0.  0.]
2	0.166663	0.030612 [ 0.14285714  0.28571429]
3	0.128203	0.070888 [ 0.2173913   0.43478261]
4	0.072449	0.113211 [ 0.27472527  0.54945055]
5	0.034001	0.141489 [ 0.30712531  0.61425061]
6	0.014562	0.155846 [ 0.3223311  0.6446622]
7	0.005991	0.162207 [ 0.32884352  0.65768705]
8	0.002424	0.164861 [ 0.33152278  0.66304555]
9	0.000974	0.165941 [ 0.33260674  0.66521349]
10	0.000390	0.166376 [ 0.33304232  0.66608463]
11	0.000156	0.166550 [ 0.33321687  0.66643373]
12	0.000063	0.166620 [ 0.33328674  0.66657347]
13	0.000025	0.166648 [ 0.33331469  0.66662939]
14	0.000010	0.166659 [ 0.33332588  0.66665175]
15	0.000004	0.166664 [ 0.33333035  0.6666607 ]
16	0.000002	0.166665 [ 0.33333214  0.66666428]
17	0.000001	0.166666 [ 0.33333286  0.66666571]
x: [ 0.33333314  0.66666628]
minf: 0.166666475779395


# 通用外罚函数法

In [22]:
def minGeneralPF(f,x0,h,cl,p,vars,eps=1e-6):
    k = 0
    FE = 0
    print 'k\ttol\tM\tx'
    for i in range(len(h)):
        FE = FE + pow(h[i],2)
    x1 = x0.T
    x2 = 1e32
    tol = 1
    while 1:
        k += 1

        M = cl*p
        FF = M*FE
        Sumf = f + FF
        
#         print 'Sumf',Sumf
        x2,minf = minNT(Sumf,x1.T,vars)
        tol = np.linalg.norm(x2-x1)
        print '%d\t%f\t%f\t%f' %(k,tol,M,fval(f,vars,x0)),
        print x1.T
        if tol <= eps:
            x = x2  
            break
        else:
            cl = M
            x1 = x2
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = t+2*s
    g= [t*s-1]
    cl = 0.05; p = 2.
    x,minf = minGeneralPF(f,np.array([2.,1]),g,cl,p,x)
    print 'x:',x
    print 'minf:',minf

k	tol	M	x
1	5.544788	0.100000	4.000000 [ 2.  1.]
2	0.556071	0.200000	4.000000 [-2.95940954 -1.47970477]
3	0.410870	0.400000	4.000000 [-2.46204479 -1.23102239]
4	0.291411	0.800000	4.000000 [-2.09455148 -1.04727574]
5	0.195199	1.600000	4.000000 [-1.83390585 -0.91695292]
6	0.121975	3.200000	4.000000 [-1.65931411 -0.82965705]
7	0.071042	6.400000	4.000000 [-1.55021599 -0.775108  ]
8	0.039028	12.800000	4.000000 [-1.48667438 -0.74333719]
9	0.020586	25.600000	4.000000 [-1.45176701 -0.72588351]
10	0.010593	51.200000	4.000000 [-1.43335446 -0.71667723]
11	0.005376	102.400000	4.000000 [-1.42387986 -0.71193993]
12	0.002709	204.800000	4.000000 [-1.41907132 -0.70953566]
13	0.001360	409.600000	4.000000 [-1.41664868 -0.70832434]
14	0.000681	819.200000	4.000000 [-1.41543269 -0.70771634]
15	0.000341	1638.400000	4.000000 [-1.41482352 -0.70741176]
16	0.000171	3276.800000	4.000000 [-1.41451864 -0.70725932]
17	0.000085	6553.600000	4.000000 [-1.41436613 -0.70718306]
18	0.000043	13107.200000	4.000000 [-1.41428

# 混合约束多维外罚函数法

In [23]:
def minConPF(f,x0,g,h,cl,p,vars,eps=1e-6):
    k = 0
    FE = 0
    print 'k\ttol\tM\tx'
    for i in range(len(h)):
        FE = FE + pow(h[i],2)
    x1 = x0.T
    x2 = 1e32
    tol = 1
    while 1:
        k += 1

        M = cl*p
        FF = M*FE
#         print g
#         print vars
        gx = matval(g,vars,x1)
        gf = 0
        for i in range(len(h)):
            if gx[i] < 0:
                gf = gf + M*(g[i]*g[i])
        Sumf = f + FF + gf
        x2,minf = minNT(Sumf,x1.T,vars)
        tol = np.linalg.norm(x2-x1)
        print '%d\t%f\t%f\t%f' %(k,tol,M,fval(f,vars,x0)),
        print x1.T
        if tol <=eps:
            x = x2
            break
        else:
            cl = M
            x1 = x2
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s - s*t + t - s +1
    g= np.array([s*s + t*t - 6,s,t]);h = np.array([2*s + 3*t -9])
    cl = 0.05; p = 2.
    x,minf = minConPF(f,np.array([2.,2]),g,h,cl,p,x)
    print 'x:',x
    print 'minf:',minf

k	tol	M	x
1	0.590593	0.100000	1.000000 [ 2.  2.]
2	0.124276	0.200000	1.000000 [ 2.28  1.48]
3	0.054033	0.400000	1.000000 [ 2.16363636  1.43636364]
4	0.025292	0.800000	1.000000 [ 2.11304348  1.4173913 ]
5	0.012247	1.600000	1.000000 [ 2.0893617   1.40851064]
6	0.006027	3.200000	1.000000 [ 2.07789474  1.40421053]
7	0.002990	6.400000	1.000000 [ 2.07225131  1.40209424]
8	0.001489	12.800000	1.000000 [ 2.0694517   1.40104439]
9	0.000743	25.600000	1.000000 [ 2.06805737  1.40052151]
10	0.000371	51.200000	1.000000 [ 2.06736156  1.40026059]
11	0.000186	102.400000	1.000000 [ 2.067014    1.40013025]
12	0.000093	204.800000	1.000000 [ 2.06684031  1.40006511]
13	0.000046	409.600000	1.000000 [ 2.06675348  1.40003255]
14	0.000023	819.200000	1.000000 [ 2.06671007  1.40001628]
15	0.000012	1638.400000	1.000000 [ 2.06668837  1.40000814]
16	0.000006	3276.800000	1.000000 [ 2.06667752  1.40000407]
17	0.000003	6553.600000	1.000000 [ 2.06667209  1.40000203]
18	0.000001	13107.200000	1.000000 [ 2.06666938  1.40000

# 内罚函数法

In [24]:
def minNF(f,x0,g,u,v,vars,eps=1e-6):
    k = 0
    FE = 0
    print 'k\ttol\tv*u\tx'
    for i in range(len(g)):
        FE = FE + 1./g[i]
    x1 = x0.T
    x2 = 1e32
    tol = 1
    while 1:
        k += 1
        FF = u*FE
        Sumf = f + FF
        x2,minf = minNT(Sumf,x1.T,vars)
        Bx = fval(FE,vars,x2)
        tol = u*Bx
        print '%d\t%f\t%f\t%f' %(k,tol,v*u,fval(f,vars,x1)),
        print x1.T
        if  np.linalg.norm(x2-x1) <= eps:
            x = x2
            break
        else:
            u = v*u
            x1 = x2
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s + 2*t*t -4*s-8*t+15
    g= np.array([9-s*s-t*t,s,t])
    x,minf = minNF(f,np.array([1.,1]),g,8.,0.5,x)
    print 'x:',x
    print 'minf:',minf

k	tol	v*u	x
1	11.926800	4.000000	6.000000 [ 1.  1.]
2	5.989363	2.000000	3.178134 [ 1.77944774  1.71566344]
3	3.022047	1.000000	3.142417 [ 1.81316664  1.73055035]
4	1.535878	0.500000	3.104449 [ 1.848716    1.75777025]
5	0.787421	0.250000	3.069664 [ 1.88206486  1.79543441]
6	0.407059	0.125000	3.042171 [ 1.91120987  1.83750765]
7	0.211655	0.062500	3.023195 [ 1.93554513  1.87799191]
8	0.110255	0.031250	3.011568 [ 1.95509738  1.91319407]
9	0.057276	0.015625	3.005192 [ 1.97017724  1.94157616]
10	0.029555	0.007812	3.002077 [ 1.98124334  1.96294449]
11	0.015119	0.003906	3.000736 [ 1.9888691   1.97789393]
12	0.007671	0.001953	3.000233 [ 1.99374758  1.98754328]
13	0.003869	0.000977	3.000068 [ 1.99664086  1.99329544]
14	0.001943	0.000488	3.000018 [ 1.9982502   1.99650409]
15	0.000974	0.000244	3.000005 [ 1.99910557  1.9982121 ]
16	0.000488	0.000122	3.000001 [ 1.99954762  1.99909548]
17	0.000244	0.000061	3.000000 [ 1.99977248  1.99954502]
18	0.000122	0.000031	3.000000 [ 1.9998859   1.99977182]
19	0

# 混合罚函数法

In [25]:
def minMixFun(f,x0,g,h,r0,c,vars,eps=1e-6):
    gx0 = np.array(matval(g,vars,x0))
    if gx0.all < 0:
        print ('The initial point must satisfy to the non-equation conditions!')
        return 0,0
    if r0 <= 0:
        print ('The initial barrier factor must be greater than 0!')
        return 0,0
    if c >= 1 or c < 0:
        print ('The scale factor must be betweem 0 and 1!')
        return 0,0
    k = 0
    FE = 0
    print 'k\ttol\tc*r0\tx'
    for i in range(len(g)):
        FE = FE + 1/g[i]
    FH = h.T * h
    x1 = x0.T
    x2 = 1e32
    tol = 1
    
    while 1:
        k += 1
        FF = r0*FE + FH/sqrt(r0)
        Sumf = f + FF
        x2,minf = minNT(Sumf[0],x1.T,vars)
        tol = np.linalg.norm(x2-x1)
        print '%d\t%f\t%f\t%f' %(k,tol,c*r0,fval(f,vars,x1)),
        print x1.T
        if  tol <= eps:
            x = x2
            break
        else:
            r0 = c*r0
            x1 = x2
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s*s - s*t + t - s +1
    g= np.array([s*s+t*t-6.,s,t]);h = np.array([2*s + 3*t-1])
    x,minf = minMixFun(f,np.array([2,2.]),g,h,2.,0.5,x)
    print 'x:',x
    print 'minf:',minf

k	tol	c*r0	x
1	2.017776	1.000000	5.000000 [ 2.  2.]
2	0.191962	0.500000	0.800906 [ 0.50223776  0.64791641]
3	0.132083	0.250000	0.806293 [ 0.3715315   0.50732674]
4	0.085580	0.125000	0.826338 [ 0.28292798  0.40937157]
5	0.050119	0.062500	0.839101 [ 0.22286591  0.34840894]
6	0.032666	0.031250	0.835240 [ 0.18106984  0.32075036]
7	0.036369	0.015625	0.811759 [ 0.14844136  0.32232292]
8	0.039123	0.007812	0.774943 [ 0.11919699  0.34394445]
9	0.035514	0.003906	0.737203 [ 0.09292354  0.37293302]
10	0.029415	0.001953	0.706067 [ 0.07085829  0.40075999]
11	0.023301	0.000977	0.682730 [ 0.05324165  0.42431645]
12	0.018001	0.000488	0.665896 [ 0.03957128  0.44318604]
13	0.013668	0.000244	0.653925 [ 0.02914688  0.4578614 ]
14	0.010237	0.000122	0.645453 [ 0.02130047  0.46905333]
15	0.007577	0.000061	0.639464 [ 0.01545977  0.47746002]
16	0.005552	0.000031	0.635230 [ 0.01115502  0.48369531]
17	0.004035	0.000015	0.632236 [ 0.00801001  0.48827115]
18	0.002912	0.000008	0.630118 [ 0.00572942  0.4915998 ]
19	0

# 混合罚函数加速法

In [26]:
def minJSMixFun(f,x0,g,h,r0,c,vars,eps=1e-6):
    gx0 = np.array(matval(g,vars,x0))
    if gx0.all < 0:
        print ('The initial point must satisfy to the non-equation conditions!')
        return 0,0
    if r0 <= 0:
        print ('The initial barrier factor must be greater than 0!')
        return 0,0
    if c >= 1 or c < 0:
        print ('The scale factor must be betweem 0 and 1!')
        return 0,0
    k = 0
    FE = 0
    print 'k\ttol\tM\tx'
    for i in range(len(g)):
        FE = FE + 1/g[i]
    FH = h.T * h
    x1 = x0.T
    FF = r0*FE + FH/sqrt(r0)
    Sumf = f + FF
    x2,minf = minNT(Sumf[0],x1.T,vars)
    tol = 1
    
    while 1:
        k += 1
        FF = r0*FE + FH/sqrt(r0)
        Sumf = f + FF
        a0 = (c*x1-x2)/(c-1)
        x2 = a0 + (x1-a0)*c*c
        x3,minf = minNT(Sumf[0],x2.T,vars)
        tol = np.linalg.norm(x3-x2)
        M = c*r0
        print '%d\t%f\t%f\t%f' %(k,tol,M,fval(f,vars,x1)),
        print x1.T
        if  tol <= eps:
            x = x3
            break
        else:
            r0 = c*r0
            x1 = x2
            x2 = x3
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s*s - s*t + t - s +1
    g= np.array([s*s+t*t-6.,s,t]);h = np.array([2*s + 3*t-1])
    x,minf = minJSMixFun(f,np.array([2,2.]),g,h,2.,0.5,x)
    print 'x:',x
    print 'minf:',minf

k	tol	M	x
1	2.814739	1.000000	5.000000 [ 2.  2.]
2	3.014198	0.500000	0.774523 [-0.24664335 -0.02812539]
3	1.499058	0.250000	7.287697 [ 3.94931431 -0.49731769]
4	0.739759	0.125000	-1.199680 [-0.49359251 -1.33559294]
5	0.382457	0.062500	4.433676 [ 1.71528195 -0.91486066]
6	0.180403	0.031250	2.022690 [ 0.62661985 -1.12959745]
7	0.098548	0.015625	3.358401 [ 1.19095438 -1.02644106]
8	0.043154	0.007812	2.744338 [ 0.92595303 -1.08122542]
9	0.026006	0.003906	3.081484 [ 1.07163549 -1.05616863]
10	0.009861	0.001953	2.931978 [ 1.00848377 -1.07037855]
11	0.007180	0.000977	3.019903 [ 1.04704415 -1.06447788]
12	0.002021	0.000488	2.985144 [ 1.03275176 -1.06828763]
13	0.002137	0.000244	3.009037 [ 1.04344331 -1.06699445]
14	0.000317	0.000122	3.001694 [ 1.0406116  -1.06807559]
15	0.000707	0.000061	3.008624 [ 1.04380798 -1.06784331]
16	0.000102	0.000031	3.007464 [ 1.04346996 -1.06817795]
17	0.000264	0.000015	3.009675 [ 1.04453053 -1.06816539]
18	0.000084	0.000008	3.009723 [ 1.04463088 -1.06828123]
19	0.0

# 乘子法

In [27]:
def minFactor(f,x0,h,v,M,alpha,gamma,vars,eps=1e-6):
    k = 0
    FE = 0
    print 'k\ttol\tM\tx'
    for i in range(len(h)):
        FE = FE + h[i]*h[i]
    x1 = x0.T
    x2 =1e32
    tol = 1
    
    while 1:
        k += 1
        FF = M*FE/2
        Fh = v*h
        Sumf = f + FF - Fh
        Sumf.tolist()
        x2,minf = minNT(Sumf[0],x1.T,vars)
        x2 = np.array(x2)
        Hx2 = np.array(matval(h,vars,x2))
        Hx1 = np.array(matval(h,vars,x1))
        tol = np.linalg.norm(Hx2)
        print '%d\t%f\t%f\t%f' %(k,tol,M,fval(f,vars,x1)),
        print x1.T
        if  tol < eps:
            x = x2
            break
        else:
            if Hx2/Hx1 > gamma:
                M = alpha * M
                x1 = x2
            else:
                v = v - M*Hx2.T
                x1 = x2
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s*s - s*t + t - s +1
    h = np.array([t+2*s-2.])
    x,minf = minFactor(f,np.array([0,0.]),h,1.,2.,3.,0.7,x)
    print 'x:',x
    print 'minf:',minf

k	tol	M	x
1	0.416667	2.000000	1.000000 [ 0.  0.]
2	0.024949	2.000000	0.870370 [ 0.75        0.83333333]
3	0.001519	2.000000	0.785812 [ 0.4081809   0.78343496]
4	0.000092	2.000000	0.791504 [ 0.42857162  0.78647393]
5	0.000006	2.000000	0.791160 [ 0.42732947  0.78628904]
6	0.000000	2.000000	0.791180 [ 0.42740504  0.78630029]
x: [ 0.42740044  0.78629961]
minf: 0.791179191511977


# 坐标轮换法

In [28]:
def minconPS(f,x0,g,delta,u,vars,eps1=1e-6,eps2=1e-6):
    n = len(vars)
    y = x0
    bmainCon = 1
    k = 0
    print 'k\ttol1\ttol2\ty'
    while bmainCon:
        k += 1
        yf = fval(f,vars,y)
        yk_1 = y
        for i in range(n):
#             print 'i:',i
            tmpy = np.zeros(np.size(y))
            #正方向搜索
            tmpy[i] = delta[i]
            tmpf = fval(f,vars,y+tmpy)
#             print '+tmpf',tmpf,'+yf',yf,tmpf<yf
            if (tmpf < yf and min([fval(g[j],vars,y+tmpy) for j in range(len(g))]) >= 0): #搜索到可行解
                bcon = 1.
                while bcon:

                    tmpy[i] = 2. * tmpy[i]   #加大步长
                    tmpf_i = fval(f,vars,y+tmpy)
                    if tmpf_i < yf and min([fval(g[j],vars,y+tmpy) for j in range(len(g))]) >=0:
                        y_res = y + tmpy
#                         print '+i:',i
#                         print '+y:',y,tmpy
                    else:
                        bcon = 0
                        
            #负方向搜索
            else:      
                tmpy[i] = delta[i]
                tmpf = fval(f,vars,y-tmpy)
#                 print '-tmpf',tmpf,'-yf',yf,tmpf<yf
                if tmpf < yf and min([fval(g[j],vars,y-tmpy) for j in range(len(g))]) >= 0: #搜索到可行解
                    bcon = 1
#                     print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
                    while bcon:
#                         print '-i:',i
                        tmpy[i] = 2. * tmpy[i] #加大步长
                        tmpf_i = fval(f,vars,y-tmpy)
                        if tmpf_i < yf and min([fval(g[j],vars,y-tmpy) for j in range(len(g))]) >= 0:
                            y_res = y - tmpy
                        else:
                            bcon = 0
                else:
                    y_res = y
                    delta = delta/u
            y = y_res
        tol1 = np.linalg.norm(y-yk_1)
        tol2 = max(np.abs(delta))
        print '%d\t%f\t%f\t%f' %(k,tol1,tol2,fval(f,vars,y)),
        print y
        if tol1 <= eps2:
            if tol2 <= eps1:
                x = y
                bmainCon = 0
            else:
                delta = delta/u
#                 print 'delta:',delta
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s + 2*t*t - 4*s - 8*t +15
    g = np.array([-s*s-t*t+9.,s,t])
    xloc,minf = minconPS(f,np.array([1,1.]),g,np.array([0.1,0.1]),1.5,x)
    print 'xloc:',xloc
    print 'minf:',minf
#     x,minf = minNF(f,np.array([1.,1.]),g,8.,0.5,x)

    f = s*s + 2*t*t -4*s-8*t+15
    g= np.array([9-s*s-t*t,s,t])
    xloc,minf = minNF(f,np.array([1.,1]),g,8.,0.5,x)   
    print 'xloc:',xloc
    print 'minf:',minf

k	tol1	tol2	y
1	1.649242	0.100000	4.080000 [ 2.6  1.4]
2	1.131371	0.100000	3.120000 [ 1.8  2.2]
3	0.447214	0.100000	3.040000 [ 2.   1.8]
4	0.266667	0.066667	3.004444 [ 2.          2.06666667]
5	0.088889	0.044444	3.000494 [ 2.          1.97777778]
6	0.000000	0.029630	3.000494 [ 2.          1.97777778]
7	0.026337	0.013169	3.000017 [ 2.          2.00411523]
8	0.000000	0.005853	3.000017 [ 2.          2.00411523]
9	0.005202	0.002601	3.000001 [ 2.          1.99891277]
10	0.000000	0.001734	3.000001 [ 2.          1.99891277]
11	0.001541	0.000771	3.000000 [ 2.          2.00045424]
12	0.000000	0.000514	3.000000 [ 2.          2.00045424]
13	0.000457	0.000228	3.000000 [ 2.          1.99999751]
14	0.000000	0.000101	3.000000 [ 2.          1.99999751]
15	0.000000	0.000030	3.000000 [ 2.          1.99999751]
16	0.000000	0.000009	3.000000 [ 2.          1.99999751]
17	0.000000	0.000004	3.000000 [ 2.          1.99999751]
18	0.000004	0.000002	3.000000 [ 2.          2.00000103]
19	0.000000	0.000001	3.000000

# 复合形法

In [29]:
def minconSimpleSearch(f,g,X,alpha,sita,gamma,beta,vars,eps=1e-6):
    n = np.size(X,1)
    FX = np.zeros(n)
    k = 0
    print 'k\ttol1\ttol2\ty'
    while 1:
        k += 1
        for i in range(n):
            FX[i] = fval(f,vars,X[:,i])
        XS = np.sort(FX)                              #对计算结果进行排序
        IX = [int(np.argwhere(FX==i)) for i in XS]    #获取序号
        Xsorted = []                                   #形成排序后的X
        for i in IX:
            Xsorted.append(list(X[:,i]))
        Xsorted = np.array(Xsorted).T
        x = Xsorted[:,0]
        
        print 'FX:',FX
        print 'Xsorted:',Xsorted
        
        px = sum(Xsorted[:,:(n-1)])/(n-1)                #计算各节点距离形心的数值差
        print 'px:',px
        Fpx = fval(f,vars,px)
        SumF = 0
        for i in range(n):
            SumF =SumF + pow((FX[IX[i]]-Fpx),2)
        SumF = sqrt(SumF/n)                            #计算各节点距离形心的数值差

        if SumF <= eps:
            x = Xsorted[:,0]
            break
        else:
            bcon_1 = 1
            cof_alpha = alpha
            k1 = 0
            while bcon_1:
                x2 = px + cof_alpha*(px - Xsorted[:,-1])
                gx2 = matval(g,vars,x2)
                if min(gx2) >= 0:
                    bcon_1 = 0
                else:
                    cof_alpha = sqrt(cof_alpha)
#                     break
                k1 +=1
                if k1 > 100:
                    break
            fx2 = fval(f,vars,x2)
            if fx2 < XS[1]:                          #小于第一个点，则在中点和数值最大点之间选点
#                 print 'here'
                cof_gamma = gamma
                bcon_2 = 1
                k2 = 0
                while bcon_2:
                    k += 1
                    print 'f3:',f
                    x3 = px + cof_gamma *(x2 - px)
                    gx3 = [fval(f,vars,x3)]
                    print 'gx3:',gx3
                    if min(gx3)>=0:
                        bcon_2 = 0
                    else:
                        cof_gamma = sqrt(cof_gamma)
                    if k2 > 100:
                        break
                fx3 = fval(f,vars,x3)
                if fx3 < XS[0]:
                    Xsorted[:,-1] = x3
                    X = Xsorted
                    continue
                else:                                    #小于第一个点，则在中点和数值最大点之间选点
                    Xsorted[:,-1] = x2
                    X = Xsorted
                    continue
            else:
                if fx2 < XS[-1]:
                    Xsorted[:,-1] = x2
                    X = Xsorted
                    continue
                else:
                    if fx2 < XS[-1]:
                        Xsorted[:,-1] = x2
                    cof_beta = beta
                    bcon_3 = 1
                    k3 = 0
                    while bcon_3:
                        print bcon_3
                        x4 = px + beta*(Xsorted[:,-1] - px)
                        gx4 = [fval(f,vars,x4)]
                        if min(gx4) >= 0:
                            bcon_3 = 0
                        else:
                            cof_beta = cof_beta/2
                        if k3 > 100:
                            break
                    fx4 = fval(f,vars,x4)
                    FNnew = fval(f,vars,Xsorted[:,-1])
                    if fx4 < FNnew:
                        Xsorted[:,-1] = x4
                        X = Xsorted
                        continue
                    else:
                        x0 = Xsorted[:,0]
                        for j in range(n):
                            Xsorted[:,j] = x0 + sita*(Xsorted[:,j]-x0)
        print '%d\t%f' %(k,fval(f,vars,x)),
        print Xsorted
        if k == 1000:
            break
            
        x = Xsorted
    minf = fval(f,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t s')
    f = s*s + 2*t*t - 4*s - 8*t +15
    g = np.array([-s*s-t*t+9.,s,t])
    X = np.array([[1,2,1.2],[2,1.5,2.5]])
    xloc,minf = minconSimpleSearch(f,g,X,1.2,0.5,8.,0.3,x)
    print 'xloc:',xloc
    print 'minf:',minf

k	tol1	tol2	y
FX: [ 5.    3.25  4.53]
Xsorted: [[ 2.   1.2  1. ]
 [ 1.5  2.5  2. ]]
px: [ 1.75  1.85]
f3: s**2 - 4*s + 2*t**2 - 8*t + 15
gx3: [70.9475000000000]
FX: [ 3.25  4.53  3.59]
Xsorted: [[ 2.   2.5  1.2]
 [ 1.5  1.7  2.5]]
px: [ 1.75  2.1 ]
f3: s**2 - 4*s + 2*t**2 - 8*t + 15
gx3: [67.5894000000000]
FX: [ 3.25    3.59    3.4806]
Xsorted: [[ 2.    2.41  2.5 ]
 [ 1.5   1.62  1.7 ]]
px: [ 1.75   2.015]
1
FX: [ 3.25        3.4806      3.00757025]
Xsorted: [[ 1.975   2.      2.41  ]
 [ 1.9205  1.5     1.62  ]]
px: [ 1.94775  1.75   ]
1
FX: [ 3.00757025  3.25        3.09845956]
Xsorted: [[ 1.975     2.086425  2.      ]
 [ 1.9205    1.711     1.5     ]]
px: [ 1.94775    1.8987125]
FX: [ 3.00757025  3.09845956  3.12273893]
Xsorted: [[ 1.975       2.086425    1.89306331]
 [ 1.9205      1.711       2.31601901]]
px: [ 1.94775    1.8987125]
1
FX: [ 3.00757025  3.09845956  3.00999872]
Xsorted: [[ 1.975       1.93134399  2.086425  ]
 [ 1.9205      2.02390445  1.711     ]]
px: [ 1.94775     1.

# 单纯形法

In [30]:
def SimpleMthd(A,c,b,baseVector):
    nVia = np.size(A,1)
    n = np.size(A,0)
    xx = range(nVia)
    nobase = []
    x = np.zeros(len(c))
    m = 1
    
    check = c>=0
    if check.all():
        vr,vr_value = find(c,'!=0',1,'right')
        rgv = A[:,(nVia-n):nVia].I*b
        check = rgv>=0
        if check.all():
            x = np.zeros(len(c))
            minf = 0
        else:
            print ('No found!')
            return -9999,-9999
    for i in range(nVia):
        if xx[i] not in baseVector:
            nobase.append(i)
        else:
            pass
    bCon = 1
    M = 0
    print ('M\tf')
    while bCon:
        nB = A[:,nobase]
        ncb = c[nobase]
        B = A[:,baseVector]
        cb = c[baseVector]
        xb = B.I*b
        f = cb*xb
        w = cb*B.I
        
        sigma = np.array([w*nB[:,i] - ncb[i] for i in range(len(nobase))])
        maxs = max(sigma)
        ind = int(np.where(sigma==maxs)[0])
        if maxs <= 0:
            minf = cb*xb
            vr,vr_value = find(c,'!=0',1,'right')
            for l in range(vr[0]+1):
                ele,ele_value = find(baseVector,'==%d' %l,1)
                if len(ele[0]) == 0:
                    x[l] = 0
                else:
                    x[l] = xb[ele[0],0]
                bCon = 0
        else:
            y = B.I*A[:,nobase[ind]]
            check = y<=0
            if check.all():
                print ('No found!')
                return -9999,-9999
            else:
                minb = 1e32
                chagB = 0
                for j in range(len(y)):
                    if y[j] > 0:
                        bz = xb[j]/y[j]
                        if bz < minb:
                            minb = bz
                            chagB = j
                tmp = baseVector[chagB]
                baseVector[chagB] = nobase[ind]
                nobase[ind] = tmp
        M = M+1
        print ('%d\t%f' %(M,f))
        if M ==10000:
            print ('No found!')
            return -9999,-9999
    return x,minf
if __name__ == '__main__':
    A = np.matrix([[-1,2,1,0,0],[2,3,0,1,0],[1,-1,0,0,1]])
    b = np.array([[4],[12],[3]])
    c = np.array([-4,-1,0,0,0])
    x,minf = SimpleMthd(A,c,b,np.array([2,3,4]))
    print 'x:',x
    print 'minf:',minf
    print '*'*60
    A = np.matrix([[-1,2,1,0],[2,3,0,1]])
    b = np.array([[4],[12]])
    c = np.array([2,1,0,0])
    x,minf = SimpleMthd(A,c,b,np.array([2,3]))
    print 'x:',x
    print 'minf:',minf
    print '*'*60
    A = np.matrix([[2,-1,1,0,0],[2,1,0,-1,0],[1,2,0,0,1]])
    b = np.array([[8],[2],[10]])
    c = np.array([1,-3,1,0,0])
    x,minf = SimpleMthd(A,c,b,np.array([2,3,4]))
    print 'x:',x
    print 'minf:',minf
    print '*'*60

M	f
1	0.000000
2	-12.000000
3	-18.000000
x: [ 4.2  1.2  0.   0.   0. ]
minf: [[-18.]]
************************************************************
M	f
1	0.000000
x: [ 0.  0.  0.  0.]
minf: [[ 0.]]
************************************************************
M	f
1	8.000000
2	-2.000000
x: [  0.   5.  13.   0.   0.]
minf: [[-2.]]
************************************************************


# 修正单纯形法

In [31]:
def ModifSimpleMthd(A,c,b,baseVector):
    nVia = np.size(A,1)
    n = np.size(A,0)
    xx = range(nVia)
    nobase = []
    x = np.zeros(len(c))
    m = 1
    
    check = c>=0
    if check.all():
        vr,vr_value = find(c,'!=0',1,'right')
        rgv = A[:,(nVia-n):nVia].I*b
        check = rgv>=0
        print check.all()
        if check.all():
            x = np.zeros(len(c))
            minf = 0
        else:
            print ('No found!')
            return -9999,-9999
    for i in range(nVia):
        if xx[i] not in baseVector:
            nobase.append(i)
        else:
            pass
    bCon = 1
    M = 0
    B = A[:,baseVector]
    invB = B.I
    print ('M\tf')
    while bCon:
        nB = A[:,nobase]
        ncb = c[nobase]
        B = A[:,baseVector]
        cb = c[baseVector]
        xb = invB*b
        f = cb*xb
        w = cb*invB
        
        sigma = np.array([w*nB[:,i] - ncb[i] for i in range(len(nobase))])
        maxs = max(sigma)
        ind = int(np.where(sigma==maxs)[0])
        if maxs <= 0:
            minf = cb*xb
            vr,vr_value = find(c,'!=0',1,'right')
            for l in range(vr[0]+1):
                ele,ele_value = find(baseVector,'==%d' %l,1)
                if len(ele[0]) == 0:
                    x[l] = 0
                else:
                    x[l] = xb[ele[0],0]
                bCon = 0
        else:
            y = B.I*A[:,nobase[ind]]
            check = y<=0
            if check.all():
                print ('No found!')
                return -9999,-9999
            else:
                minb = 1e32
                chagB = 0
                for j in range(len(y)):
                    if y[j] > 0:
                        bz = xb[j]/y[j]
                        if bz < minb:
                            minb = bz
                            chagB = j
                tmp = baseVector[chagB]
                baseVector[chagB] = nobase[ind]
                nobase[ind] = tmp
            for j in range(chagB):
                if y[j][0,0] != 0:
                    invB[j,:] = invB[j,:] -invB[chagB,:]*y[j][0,0]/y[chagB][0,0]
            for j in range(chagB+1,len(y),1):
                if y[j][0,0] != 0:
                    invB[j,:] = invB[j,:] -invB[chagB,:]*y[j][0,0]/y[chagB][0,0]
            invB[chagB,:] = invB[chagB,:]/y[chagB][0,0]
        M = M+1
        print ('%d\t%f' %(M,f))
        if M ==10000:
            print ('No found!')
            return -9999,-9999
    return x,minf
if __name__ == '__main__':
    A = np.matrix([[-1,2,1,0,0],[2,3,0,1,0],[1,-1,0,0,1]])
    b = np.array([[4],[12],[3]])
    c = np.array([-4,-1,0,0,0])
    x,minf = ModifSimpleMthd(A,c,b,np.array([2,3,4]))
    print 'x:',x
    print 'minf:',minf
    print '*'*60
    
    A = np.matrix([[-1,-2,-1,0],[2,3,0,1]])
    b = np.array([[4],[12]])
    c = np.array([2.,1,0,0])
    x,minf = ModifSimpleMthd(A,c,b,np.array([2,3]))
    print 'x:',x
    print 'minf:',minf
    print '*'*60
    
    A = np.matrix([[2,-1,1,0,0],[2,1,0,-1,0],[1,2,0,0,1]])
    b = np.array([[8],[2],[10]])
    c = np.array([1,-3,1,0,0])
    x,minf = ModifSimpleMthd(A,c,b,np.array([2,3,4]))
    print 'x:',x
    print 'minf:',minf
    print '*'*60

M	f
1	0.000000
2	-12.000000
3	-18.000000
x: [ 4.2  1.2  0.   0.   0. ]
minf: [[-18.]]
************************************************************
False
No found!
x: -9999
minf: -9999
************************************************************
M	f
1	8.000000
2	-2.000000
x: [  0.   5.  13.   0.   0.]
minf: [[-2.]]
************************************************************


# G-N法

In [32]:
def minGN(f,x0,vars,eps=1e-6):
    
    A = jacobian(f,vars)
    f = np.array(f)
    S = np.dot( np.transpose(f) , f)
    print S
    k = len(f)
    n = len(x0)
    x0 = x0.T
    tol=1.
    K = 0
    print ('K\ttol\tminf\tx\t')
    while tol>eps:
        K += 1
        Fx = np.matrix([fval(f[i],vars,x0) for i in range(k)])
#         print 'Fx',Fx
        Sx = fval(S,vars,x0)
#         print 'Sx',Sx
        Ax = np.matrix([fval(A[i],vars,x0) for i in range(len(A))])
#         print 'Ax',Ax
        gSx = np.dot(Fx,np.transpose(Ax))
#         print 'gSx:',gSx
        dx = np.dot(-np.dot(Ax,np.transpose(Ax)),1./gSx)
#         dx = np.dot(-1/gSx,np.dot(Ax,Ax.T))
#         print 'dx',dx[0][0],type(dx[0][0])
#         print 'x0',x0
        x0[0] += dx[0][0]
#         print 'x0',x0
        tol = np.linalg.norm(float(dx))
#         print K
        print ('%d\t%f\t%f' %(K,tol,fval(S,vars,x0))),
        print x0
        if K == 1000:
            break
    x = x0

    minf = fval(S,vars,x)
    return x,minf
if __name__ == '__main__':
    x = var('t')
    fx = [t+1,0.5*t*t +t -1]
    x,minf = minGN(fx,np.array([1.]),x)
    print 'x:',x
    print 'minf:',minf
            

(t + 1)**2 + (0.5*t**2 + t - 1)**2
K	tol	minf	x	
1	1.666667	2.197531 [-0.66666667]
2	7.500000	912.865934 [ 6.83333333]
3	0.263779	794.370557 [ 6.56955446]
4	0.273603	684.017057 [ 6.29595179]
5	0.284621	581.816441 [ 6.01133053]
6	0.297099	487.781576 [ 5.71423163]
7	0.311390	401.927711 [ 5.40284198]
8	0.327981	324.273204 [ 5.07486121]
9	0.347565	254.840581 [ 4.72729633]
10	0.371166	193.658142 [ 4.35613018]
11	0.400376	140.762498 [ 3.95575413]
12	0.437831	96.202837 [ 3.51792311]
13	0.488291	60.048654 [ 3.02963192]
14	0.561466	32.405146 [ 2.46816555]
15	0.681256	13.447621 [ 1.78690989]
16	0.929745	3.499475 [ 0.85716489]
17	1.956358	2.245105 [-1.09919277]
18	20.563473	45359.449829 [-21.6626661]
19	0.097247	44509.591056 [-21.56541869]
20	0.097712	43667.751244 [-21.46770708]
21	0.098183	42833.930573 [-21.36952456]
22	0.098660	42008.129229 [-21.27086424]
23	0.099145	41190.347401 [-21.17171907]
24	0.099637	40380.585282 [-21.07208182]
25	0.100137	39578.843068 [-20.97194507]
26	0.100644	38785.120

# 粒子群法（PSO）

In [33]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 0
    if type(x) == list:
        for i in range(len(x)):
            F = F + x[i]*x[i]
    else:
        F = F + x*x
    return F

def PSO(N,c1,c2,w,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    Pbest = []
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i]-x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
        Pbest.append(fitness(pg))
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = PSO(40,2,2,0.5,1000,30)
    print 'minf:',minf
    print '#'*60
    xv,minf = PSO(40,2,2,0.5,5000,30)
    print 'minf:',minf
    print '#'*60
    xv,minf = PSO(40,2,2,0.5,10000,30)
    print 'minf:',minf
    print '#'*60

minf: 0.832886146539
############################################################
minf: 0.0786073636093
############################################################
minf: 0.0437211187982
############################################################


# 压缩粒子群法

In [34]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 0
    if type(x) == list:
        for i in range(len(x)):
            F = F + x[i]*x[i] - np.cos(18.*x[i])
    return F

def YAPSO(N,c1,c2,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    phi = c1 + c2
    if phi <=4:
        print 'phi should be greater than the sum of c1 and c2!'
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        for i in range(N):
            ksi = 2./abs(2-phi-np.sqrt(phi*phi - 4*phi))
            v[i] = v[i] + c1*np.random.rand()*(y[i]-x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            v[i] = ksi*v[i]
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = YAPSO(40,2.05,2.05,10000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60


xv: [-6.166267883134053e-11, 9.232114291524741e-10]
minf: -2.0
############################################################


# 权重改进的粒子群法

In [35]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 100*pow(x[0]*x[0]-x[1],2) + pow(1-x[0],2)
    return F

def LinPSO(N,c1,c2,wmax,wmin,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        w = wmax - (t-1)*(wmax-wmin)/(M-1)
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i]-x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = LinPSO(40,2,2,0.9,0.4,1000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60

xv: [1.0, 1.0]
minf: 0.0
############################################################


In [36]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 100*pow(x[0]*x[0]-x[1],2) + pow(1-x[0],2)
    return F

def LinPSO(N,c1,c2,wmax,wmin,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        w = wmax - (t-1)*(wmax-wmin)/(M-1)
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i]-x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = LinPSO(40,2,2,0.9,0.4,1000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60

xv: [1.0, 1.0]
minf: 0.0
############################################################


# 随机权重

In [37]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 100*pow(x[0]*x[0]-x[1],2) + pow(1-x[0],2)
    return F

def RandPSO(N,c1,c2,mean_max,mean_min,sigma,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        for i in range(N):
            miu = mean_min + (mean_max - mean_min)*np.random.rand()
            w = miu+sigma*np.random.rand()
            v[i] = w*v[i] + c1*np.random.rand()*(y[i]-x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = RandPSO(40,2,2,0.8,0.5,0.2,10000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60

xv: [1.0, 1.0]
minf: 0.0
############################################################


# 同步学习因子粒子群法

In [38]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    try:
        F = 4*x[0]*x[0] - 2.1*pow(x[0],4) + pow(x[0],6)/3 + x[0]*x[1]-4*x[1]*x[1] + 4*pow(x[1],4)
    except:
        F = 1e32
    return F

def LnCPSO(N,cmax,cmin,w,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        c = cmax - (cmax - cmin)*t/M
        for i in range(N):
            v[i] = w*v[i] + c*np.random.rand()*(y[i]-x[i]) + c*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = LnCPSO(40,2.1,0.8,0.9,10000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60

xv: [0.0898420063844049, -0.7126564024120043]
minf: -1.03162845349
############################################################


# 二阶粒子群法

In [39]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    try:
        F = 4*x[0]*x[0] - 2.1*pow(x[0],4) + pow(x[0],6)/3 + x[0]*x[1]-4*x[1]*x[1] + 4*pow(x[1],4)
    except:
        F = 1e32
    return F

def SecPSO(N,w,c1,c2,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    xl = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i]-2*x[i]+xl[i]) + c2*np.random.rand()*(pg-2*x[i]+xl[i])   #计算粒子速度
            xl[i] = x[i]
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = SecPSO(40,0.7,1,1,5000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60


xv: [0.0898420138873171, -0.7126564025541136]
minf: -1.03162845349
############################################################


# 二阶震荡粒子群法

In [40]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    try:
        F = 4*x[0]*x[0] - 2.1*pow(x[0],4) + pow(x[0],6)/3 + x[0]*x[1]-4*x[1]*x[1] + 4*pow(x[1],4)
    except:
        F = 1e32
    return F

def SecVibratPSO(N,w,c1,c2,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    xl = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    xl = np.array(xl)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        for i in range(N):
            phi1 = c1*np.random.rand()
            phi2 = c2*np.random.rand()
            if t < M/2 :
                ks1 = (2*np.sqrt(phi1)-1)*np.random.rand()/phi1
                ks2 = (2*np.sqrt(phi2)-1)*np.random.rand()/phi2
            else:
                ks1 = (2*np.sqrt(phi1)-1)*(1+np.random.rand())/phi1
                ks2 = (2*np.sqrt(phi2)-1)*(1+np.random.rand())/phi2
            v[i] = w*v[i] + phi1*(y[i]-(1+ks1)*x[i]+ks1*xl[i]) + phi2*(pg-(1+ks2)*x[i]+ks1*xl[i])   #计算粒子速度
            xl[i] = x[i]
            x[i] += v[i] #计算粒子位移
            #如果一个粒子在新位移下的数值小于该粒子上一位置，保留新的数值p[i],并更新该粒子的最新位置y[i]
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = SecVibratPSO(40,0.78,1,1,5000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60


C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: RuntimeWarning: overflow encountered in add
C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: RuntimeWarning: invalid value encountered in add
C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: RuntimeWarning: overflow encountered in multiply


xv: [0.08981661521736813, -0.7126398196521754]
minf: -1.03162844914
############################################################


# 混沌粒子群法

In [41]:
import numpy as np
# def fitness(x):
#     '''
#     待优化的目标函数
#     '''
#     F = 0
#     for i in range(len(x)):
#         F = F + 1/(i+ (x[i]-1)*(x[i]-1))
#     return 1/(0.01+F)
def fitness(x):
    '''
    待优化的目标函数
    '''
    try:
        F = 4*x[0]*x[0] - 2.1*pow(x[0],4) + pow(x[0],6)/3 + x[0]*x[1]-4*x[1]*x[1] + 4*pow(x[1],4)
    except:
        F = 1e32
    return F

def CLSPSO(N,w,c1,c2,xmax,xmin,M,MaxC,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        fv = []
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i] - x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            fv.append(fitness(x[i].tolist()))
#####################
        sort_fv,index = ArrSort(fv)
        Nbest = int(np.floor(N*0.2))
        for n in range(Nbest):
            tmpx = x[index[n]]
            for k in range(MaxC):
                cx = []
                for dim in range(D):
                    cx.append((tmpx[dim] - xmin[dim]) / (tmpx[dim] - xmax[dim]))
                    cx[dim] = 4*cx[dim]*(1-cx[dim])
                    tmpx[dim] += cx[dim]*(xmax[dim]-xmin[dim])
                fcs = fitness(tmpx)
                if fcs < sort_fv[n]:
                    x[index[n]] = tmpx
                    break
            x[index[n]] = tmpx
        r = np.random.rand()
        for s in range(D):
            xmin[s] = max(xmin[s],pg[s] - r*(xmax[s]-xmin[s]))
            xmax[s] = min(xmax[s],pg[s] + r*(xmax[s]-xmin[s]))
        x[:Nbest] = x[index[:Nbest]]
###########################################################################
        for i in range(Nbest,N,1):
            for j in range(D):
                x[i,j] = xmin[j] + np.random.rand()*(xmax[j] - xmin[j])
                v[i,j] = np.random.randn()
        for i in range(N):
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = CLSPSO(50,0.8,2,2,np.array([10,10,10,10,10]),-np.array([10,10,10,10,10]),9000,10,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60


xv: [-0.0901268096522746, 0.712364983220689]
minf: -1.03162735912
############################################################


# 基于自然选择算法

In [42]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 0
    for i in range(len(x)):
        F = F + x[i]*x[i] - np.cos(18.*x[i])
    return F


def SelPSO(N,c1,c2,w,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入主循环，按照公式依次迭代
    for t in range(M):
        fx =[]
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i] - x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            fx.append(fitness(x[i].tolist()))

            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
        sortx,sortx = ArrSort(fx)
        exIndex = np.round((N)/2)
        x[sortx[exIndex:]] = x[sortx[:(exIndex)]]
        v[sortx[exIndex:]] = v[sortx[:(exIndex)]]
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = SelPSO(40,2,2,0.7,10000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60

xv: [-3.909394968625922e-10, 1.2497305896487923e-10]
minf: -2.0
############################################################


# 基于杂交的算法

In [43]:
import numpy as np
def fitness(x):
    '''
    待优化的目标函数
    '''
    F = 0
    for i in range(len(x)):
        F = F + x[i]*x[i] - np.cos(18.*x[i])
    return F


def BreedPSO(N,c1,c2,w,Pc,Sp,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    #######初始化结束###################
    
    #进入主循环，按照公式依次迭代
    for t in range(M):
        for i in range(N):
            v[i] = w*v[i] + c1*np.random.rand()*(y[i] - x[i]) + c2*np.random.rand()*(pg-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
#################################################################
            r1 = np.random.rand()
            if r1 < Pc:
                numPool = int(np.round(Sp*N))
                PoolX = x[:numPool]
                PoolVX = v[:numPool]
                childx1 = []
                childv1 = []
                for j in range(numPool):
                    seed1 = int(np.floor(np.random.rand()*(numPool - 1)+1))
                    seed2 = int(np.floor(np.random.rand()*(numPool - 1)+1))
                    pb = np.random.rand()
                    childx1.append(pb*PoolX[seed1] + (1-pb)*PoolX[seed2])
                    childv1.append((PoolVX[seed1] + PoolX[seed2]) * np.linalg.norm(PoolVX[seed1])/np.linalg.norm(PoolVX[seed1] + np.linalg.norm(PoolVX[seed2])))
                x[:numPool] = childx1
                v[:numPool] = childv1
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = BreedPSO(50,2,2,0.7,0.9,0.2,50,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60

xv: [-3.8385990216938115e-05, -0.3469056267345655]
minf: -1.87890035747
############################################################


# 基于模拟退火

In [44]:
import numpy as np
# def fitness(x):
#     '''
#     待优化的目标函数
#     '''
#     F = 0
#     for i in range(len(x)):
#         F = F + 1/(i+ (x[i]-1.)*(x[i]-1.))
#     return 1/(0.01+F)
def fitness(x):
    '''
    待优化的目标函数
    '''
    try:
        F = 4*x[0]*x[0] - 2.1*pow(x[0],4) + pow(x[0],6)/3 + x[0]*x[1]-4*x[1]*x[1] + 4*pow(x[1],4)
    except:
        F = 1e32
    return F

def SimuAPSO(N,c1,c2,lamda,M,D):
    '''
    粒子数目：N
    学习因子1：c1
    学习因子2：c2
    惯性权重：w
    最大迭代次数：M
    自变量数量：D
    目标函数去最小值时的自变量数目：xm
    目标函数最小值fv
    '''
    ##########初始化过程#############
    #计算各个粒子的适应度，并初始化Pi和Pg
    x = np.random.randn(N,D).tolist()
    v = np.random.randn(N,D).tolist()
    #初始化p，计算各个粒子的位置和速度
    p = [fitness(x[i]) for i in range(N)]
    y = [x[i] for i in range(N)]
    
    #初始化最优的粒子
    pg = x[N-1]                   #初选x[N-1]作为最优的粒子
    for i in range(N-1):
        if fitness(x[i]) < fitness(pg):
            pg = x[i]
    x = np.array(x)
    y = np.array(y)
    v = np.array(v)
    T = fitness(pg)/np.log(5)
    #######初始化结束###################
    
    #进入祝循环，按照公式依次迭代
    for t in range(M):
        groupFit = fitness(pg)
        Tfit=  [np.exp(-(p[i] - groupFit)/T) for i in range(N)]
        SumTfit = np.sum(Tfit)
        Tfit = Tfit/SumTfit
        pBet = np.random.rand()
        ComFit = []
        for j in range(N):
            ComFit.append(np.sum(Tfit[:j]))
            if pBet <= ComFit[j]:
                pg_plus = x[j]
                break
        C = c1+c2
        ksi = 2/abs(2-C-np.sqrt(C*C - 4*C))
        for i in range(N):
            v[i] = ksi*v[i] + c1*np.random.rand()*(y[i] - x[i]) + c2*np.random.rand()*(pg_plus-x[i])   #计算粒子速度
            x[i] += v[i] #计算粒子位移
#####################
            if fitness(x[i].tolist()) < p[i]:
                p[i] = fitness(x[i].tolist())
                y[i] = x[i]
            if p[i] <fitness(pg):
                pg = y[i].tolist()
        T = T*lamda
    xm = pg
    fv = fitness(pg)
    return xm,fv
if __name__ == '__main__':
    xv,minf = SimuAPSO(40,2.05,2.05,0.5,10000,2)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60


C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:53: RuntimeWarning: overflow encountered in exp
C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:55: RuntimeWarning: invalid value encountered in divide
C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:53: RuntimeWarning: overflow encountered in double_scalars
C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:53: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\chaos\Anaconda2\lib\site-packages\ipykernel\__main__.py:53: RuntimeWarning: invalid value encountered in double_scalars


xv: [0.08947291476117743, -0.7126283475608102]
minf: -1.03162792624
############################################################


# 遗传算法

In [2]:
def fitness(x):
    return pow(x,3)-60*pow(x,2) + 900*x + 100
def Initial(length):
    result = [round(np.random.rand()) for i in range(length)]
    return result
def Dec(a,b,x,L):
    base = np.array([pow(2,i) for i in range(L-1,-1,-1)])
    y = np.dot(base,x)
    y = a + y*(b-a)/(2*L-1)
    return y

def myGA(a,b,NP,NG,Pc,Pm,eps=1e-6):
    L = int(np.ceil(np.log2((b-a)/eps+1)))
    x = np.zeros([NP,L])
    x = [Initial(L) for i in range(NP)]
    fx = [fitness(Dec(a,b,x[i],L)) for i in range(NP)]
    PPx = []
    for k in range(NG):
        sumfx = sum(fx)
        Px = fx/sumfx
         
        for i in range(NP):
            if i == 0:
                PPx.append(Px[0])
            else:
                PPx.append(PPx[i-1] + Px[i])
        for i in range(NP):
            sita = np.random.rand()
            for n in range(NP):
                if sita <= PPx[n]:
                    SelFather = n
                    break
            Selmother = np.floor(np.random.rand() * (NP-1)) + 1
            posCut = int(np.floor(np.random.rand()*(L-2)) + 1)
            r1 = np.random.rand()

            if r1 <= Pc:
                print type(SelFather)
                print type(posCut)
                nx_temp = x[SelFather,:posCut]
                nx_temp.extend(x[Selmother,(posCut+1):L])
                nx.append[nx_temp]
                r2 = np.random.rand()
                if r2 <= Pm:
                    posMut = np.round(np.random.rand()*(L-1) + 1)
                    nx[i,posMut] = ~nx[i,posMut]
            else:
                nx[i] = x[SelFather]
        x = nx
        for i in range(NP):
            fx[i] = fitness(Dec(a,b,x[i],L))
    fv = -1e32
    for i in range(NP):
        fitx = fitness(Dec(a,b,x[i],L))
        if fitx > fv:
            fv = fitx
            xv = Dec(a,b,x[i],L)
    return xv,fv
if __name__ == '__main__':
    xv,minf = myGA(0,30,50,100,0.9,0.04,0.01)
    print 'xv:',xv
    print 'minf:',minf
    print '#'*60
                
    

<type 'int'>
<type 'int'>


TypeError: list indices must be integers, not tuple